<a href="https://colab.research.google.com/github/leonardogonoli/machine_learning_projects/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Book Recommendation Engine using KNN**

This is a challenge from the website [freeCodeCamp](https://www.freecodecamp.org/learn/machine-learning-with-python/machine-learning-with-python-projects/book-recommendation-engine-using-knn).

The goal of this challenge is to create a book recommendation algorithm using K-Nearest Neighbors.

For this challenge, I need to use the Book-Crossings[Book-Crossings](http://www2.informatik.uni-freiburg.de/~cziegler/BX/) dataset. This dataset contains 1.1 million ratings (scale of 1-10) of 270,000 books by 90,000 users.

After importing and cleaning the data, I should use NearestNeighbors from sklearn.neighbors to develop a model that shows books that are similar to a given book. The Nearest Neighbors algorithm measures the distance to determine the “closeness” of instances.

**Goal:** Create a function named get_recommends that takes a book title (from the dataset) as an argument and returns a list of 5 similar books with their distances from the book argument.

The data returned from get_recommends() is a list. The first element in the list is the book title passed into the function. The second element in the list is a list of five more lists. Each of the five lists contains a recommended book and the distance from the recommended book to the book passed into the function.

In [1]:
# import libraries
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2023-01-26 03:58:25--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  60.4MB/s    in 0.4s    

2023-01-26 03:58:25 (60.4 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [14]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
df_books.head()

         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  
(271379, 3)


In [ ]:
df_books.shape

In [15]:
df_ratings.head()

     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0
(1149780, 3)


In [ ]:
df_ratings.shape

In [16]:
# delete users with less than 200 ratings and books with less than 100 ratings
user_counts = df_ratings['user'].value_counts()
isbn_counts = df_ratings['isbn'].value_counts()

df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]
df_ratings = df_ratings[df_ratings['isbn'].isin(isbn_counts[isbn_counts >= 100].index)]
df_ratings.head()

        user        isbn  rating
1456  277427  002542730X    10.0
1469  277427  0060930535     0.0
1471  277427  0060934417     0.0
1474  277427  0061009059     9.0
1484  277427  0140067477     0.0
(49781, 3)


In [ ]:
df_ratings.shape

In [17]:
df_merged_books_ratings = df_books.merge(df_ratings, on='isbn')
df_merged_books_ratings.head()

         isbn          title        author    user  rating
0  0440234743  The Testament  John Grisham  277478     0.0
1  0440234743  The Testament  John Grisham    2977     0.0
2  0440234743  The Testament  John Grisham    3363     0.0
3  0440234743  The Testament  John Grisham    7346     9.0
4  0440234743  The Testament  John Grisham    9856     0.0
(49517, 5)


In [28]:
# combine two or more ratings from the same user for the same book into one rating
df_grouped_books_authors = df_merged_books_ratings.groupby(by = ['title', 'user'], as_index=False).agg({'rating':'mean'}).drop_duplicates()
df_grouped_books_authors.head()

  title   user  rating
0  1984    254     9.0
1  1984  11676     0.0
2  1984  12538     0.0
3  1984  15408     9.0
4  1984  16795     8.0
(49136, 3)


In [49]:
pivot_users_titles = df_grouped_books_authors.pivot_table(index = 'title', columns = 'user', values = 'rating').fillna(0)
pivot_users_titles.head()

(673, 888)

In [52]:
mat_sparse = csr_matrix(pivot_users_titles.values)
mat_sparse.shape

(673, 888)

In [53]:
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')

In [65]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  model_knn.fit(sparse_pop_ratings)
  n_neighbors = 6
  distances, indices = model_knn.kneighbors([pivot_users_titles.loc[book]], n_neighbors  = n_neighbors)
  recommends = list(zip(indices, distances))[0]
  list_of_recommendations = []
  list_of_recommendations.append(book)
  other_recommendations = []
  for i in range(n_neighbors - 1, 0, -1):
    title = pivot_users_titles.index[recommends[0][i]]
    other_recommendations.append([title, round(recommends[1][i],2)])
  list_of_recommendations.append(other_recommendations)
  return list_of_recommendations


In [48]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8], ['The Weight of Water', 0.77], ['The Surgeon', 0.77], ['I Know This Much Is True', 0.77], ['The Lovely Bones: A Novel', 0.72]]]
You passed the challenge! 🎉🎉🎉🎉🎉
